`E-COMMERCE ANALYTICS DASHBOARD`

Database improvements → SQLite को normalize + indexing करना।

Functions rename + modularization → अपने naming conventions से।

Extra analytics → top categories, seasonal sales, avg order value।

Export multiple formats → CSV + PDF download।

Better UI → Gradio dashboard में अलग-अलग tabs।

In [5]:
# E-COMMERCE ANALYTICS DASHBOARD


# Required libraries
# pandas (data handling), numpy (random simulation), sqlite3 (lightweight DB),
# flask (API routes), gradio (UI), plotly (interactive graphs), reportlab (PDF export)
!pip install pandas numpy plotly flask gradio sqlalchemy reportlab --quiet

import threading, sqlite3, io
from flask import Flask, send_file
import pandas as pd, numpy as np
import plotly.express as px
import gradio as gr
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas


# 1. Database Setup
# Dummy database jisme random sales data hai
# Real project me ye DB e-commerce system se connect hoga

conn = sqlite3.connect("ecom.db", check_same_thread=False)
cur = conn.cursor()

# Purana table delete karo agar exist kare
cur.execute("DROP TABLE IF EXISTS sales_orders")

# Naya table banaya: (product, category, qty, revenue, date)
cur.execute("""
CREATE TABLE sales_orders (
    id INTEGER PRIMARY KEY,
    product TEXT,
    category TEXT,
    quantity INT,
    revenue REAL,
    order_date TEXT
)
""")

# Categories aur products define
categories = ['Electronics','Home','Fashion','Beauty']
products_map = {
    'Electronics': ['Phone','Headset','Charger'],
    'Home': ['Lamp','Rug','Mug'],
    'Fashion': ['Tshirt','Sneakers'],
    'Beauty': ['Lotion','Perfume']
}

# Random data generate
rows=[]
np.random.seed(42)
for i in range(2000):
    cat = np.random.choice(categories, p=[0.35,0.25,0.25,0.15])
    prod = np.random.choice(products_map[cat])
    qty = np.random.randint(1,5)
    rev = round(qty * (20 + np.random.rand()*250),2)
    date = (pd.Timestamp('2023-01-01') + pd.Timedelta(days=int(np.random.rand()*730))).strftime('%Y-%m-%d')
    rows.append((prod,cat,qty,rev,date))

cur.executemany("INSERT INTO sales_orders (product,category,quantity,revenue,order_date) VALUES (?,?,?,?,?)", rows)
conn.commit()

# ==========================
# 2. Helper Functions
# ==========================

def fetch_top_products(limit=10):
    """
    Function: Returns top products by revenue
    Ye function database se sales data read karta hai
    aur sabse zyada revenue generate karne wale products dikhata hai
    """
    df = pd.read_sql("SELECT * FROM sales_orders", conn, parse_dates=['order_date'])
    summary = df.groupby('product').agg(
        total_sales=('revenue','sum'),
        total_units=('quantity','sum')
    ).reset_index().sort_values('total_sales', ascending=False)
    return summary.head(limit)

def fetch_category_trend(chosen_category='All'):
    """
    Function: Revenue trend month-wise
    Agar 'All' hai to sab category ka trend,
    warna specific category ka trend dikhayega
    """
    df = pd.read_sql("SELECT * FROM sales_orders", conn, parse_dates=['order_date'])
    if chosen_category!='All':
        df = df[df.category==chosen_category]
    df['month'] = df.order_date.dt.to_period('M').astype(str)
    monthly = df.groupby(['month','category']).revenue.sum().reset_index()
    fig = px.line(monthly, x='month', y='revenue', color='category',
                  title='Monthly Revenue by Category')
    return fig

def export_top_products_pdf():
    """
    Function: Export Top Products in PDF
    'Reporting feature added'
    """
    df = fetch_top_products(20)
    filename = "top_products.pdf"
    c = canvas.Canvas(filename, pagesize=letter)
    c.drawString(200, 750, "E-commerce: Top Products Report")
    y = 700
    for _, row in df.iterrows():
        c.drawString(100, y, f"{row['product']} - Units: {row['total_units']} - Revenue: {row['total_sales']}")
        y -= 20
    c.save()
    return filename

# ==========================
# 3. Flask API (CSV Download)
# ==========================
app = Flask("ecom_api")

@app.route("/download/top_products.csv")
def dl_top_csv():
    df = fetch_top_products(50)
    buf = io.BytesIO()
    df.to_csv(buf, index=False)
    buf.seek(0)
    return send_file(buf, mimetype='text/csv', download_name='top_products.csv')

@app.route("/download/top_products.pdf")
def dl_top_pdf():
    file = export_top_products_pdf()
    return send_file(file, mimetype='application/pdf', download_name='top_products.pdf')

def run_flask():
    app.run(host="0.0.0.0", port=7866) # Changed port to 7866

# ==========================
# 4. Gradio Dashboard
# ==========================
with gr.Blocks() as dashboard:
    gr.Markdown("##E-commerce Product Analytics Dashboard")

    # Category filter
    cat_dropdown = gr.Dropdown(["All"]+categories, value="All", label="Choose Category")

    # Buttons
    show_btn = gr.Button("Show Trend")
    pdf_btn = gr.Button("Export Top Products PDF")

    # Outputs
    plot_out = gr.Plot()
    table_out = gr.Dataframe()
    pdf_msg = gr.Textbox(label="PDF Export Status")

    # Action functions
    def show_analysis(cat):
        return fetch_category_trend(cat), fetch_top_products(10)

    def save_pdf():
        file = export_top_products_pdf()
        return f"PDF saved as {file}"

    show_btn.click(show_analysis, inputs=cat_dropdown, outputs=[plot_out, table_out])
    pdf_btn.click(save_pdf, outputs=pdf_msg)

    gr.Markdown("### Downloads:")
    gr.Markdown("- [Download Top Products CSV](http://localhost:7866/download/top_products.csv)") # Updated port in markdown link
    gr.Markdown("- [Download Top Products PDF](http://localhost:7866/download/top_products.pdf)") # Updated port in markdown link

# Run servers (Flask + Gradio)
threading.Thread(target=run_flask, daemon=True).start()
dashboard.launch(server_name="0.0.0.0", server_port=7867, share=False) # Changed port to 7867

 * Serving Flask app 'ecom_api'
 * Debug mode: off


Address already in use
Port 7866 is in use by another program. Either identify and stop that program, or start the server with a different port.


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>